In [26]:
import os;

import sys;
sys.path.insert(0,os.getcwd() + '\\..\\lib');
sys.path.insert(0,os.getcwd());
# import inspect

from Transform import GenTransform;
from Regression import LogisticRegression;



In [27]:
import re;
import pandas as pd;
import numpy as np;
import inspect;

reviewsDf= pd.read_csv(os.getcwd() +"\\datasets\\training.csv", index_col=0, parse_dates=True);
# reviewsDf
wordFreq = dict();
words = [];
stopWords = ["a","about","above","after","again","against","all","am","an","and","any","are","as","at",
            "be","because","been","before","being","below","between","both","but","by","could","did",
            "do","does","doing","down","during","each","few","for","from","further","had","has","have",
            "having","he","he'd","he'll","he's","her","here","here's","hers","herself","him","himself",
            "his","how","how's","i","i'd","i'll","i'm","i've","if","in","into","is","it","it's","its",
            "itself","let's","me","more","most","my","myself","of","off","on","once","only","or","other",
            "ought","our","ours","ourselves","out","over","own","same","she","she'd","she'll","she's",
            "should","so","some","such","than","that","that's","the","their","theirs","them","themselves",
            "then","there","there's","these","they","they'd","they'll","they're","they've","this","those",
            "through","to","too","under","until","up","very","was","we","we'd","we'll","we're","we've",
            "were","what","what's","when","when's","where","where's","which","while","who","who's","whom",
            "why","why's","with","would","you","you'd","you'll","you're","you've","your","yours","yourself",
            "yourselves", "htc", "iphone", "windows"];

reviews = reviewsDf.values[:,1];


for review in reviews:
    review = re.sub('[^0-9a-zA-Z ]+', '', review);
    reviewWords = review.lower().strip().split(" ");
    for word in reviewWords:
        if(not word in stopWords):
            if(not word in wordFreq):
                wordFreq[word] = 1;
                words.append(word);
                
            else:
                wordFreq[word] += 1;
wordFreq.pop('', None);
if '' in words: words.remove('');
    
wordFreq = sorted(wordFreq.items(), key=lambda x:x[1], reverse = True);
count = 0;
for key in wordFreq:
    if key in words: words.remove(key);
    count = count + 1;
#     if(count==100):
#         break;
        
for word in words:
    reviewsDf["is_" + word] = reviewsDf["Review"].str.lower().str.contains(word) * 1

reviewsDf = GenTransform.generateValueColumns(reviewsDf, ["Sentiment"]);
reviewsDf.columns
reviewsXColumns = [col for col in reviewsDf.columns if col not in ['positive', 'negative', 'neutral', 'ProductName', 'Review']]
reviewsX = np.matrix(reviewsDf[reviewsXColumns]);



sentiments = ["positive", "negative", "neutral"];
sentimentTheta = [];
i =0;

for sentiment in sentiments:
    reviewsy = np.transpose(np.matrix(reviewsDf[sentiment]));
    reviewLrIns = LogisticRegression(reviewsX, reviewsy, 0.5);
    theta, fminOutput = reviewLrIns.buildModel();
    sentimentTheta.append(theta);
    i = i+1;

         Current function value: 0.250319
         Iterations: 50
         Function evaluations: 119
         Gradient evaluations: 107
         Current function value: 0.239073
         Iterations: 34
         Function evaluations: 150
         Gradient evaluations: 138
         Current function value: 0.289704
         Iterations: 38
         Function evaluations: 131
         Gradient evaluations: 119


In [28]:
import MongoReviews;

# testDf= pd.read_csv(os.getcwd() +"\\datasets\\testing.csv", index_col=0, parse_dates=True);
testDf = MongoReviews.getReviews();

# print(inspect(NewReviews.getNewReviews));

for word in words:
    testDf["is_" + word] = testDf["Review"].str.lower().str.contains(word) * 1;
testX = np.matrix(testDf[reviewsXColumns]);  

predictionsSentiment = np.ones((len(testDf.index),1)) * -1;
for theta in sentimentTheta:
    predictedY = np.round(LogisticRegression.sigmoid(np.dot(LogisticRegression.addIntercept(testX),theta)));
    predictionsSentiment = np.hstack([predictionsSentiment, predictedY]);

sentimentIndices = np.subtract(np.argmax(predictionsSentiment, axis = 1), 1).flatten().tolist()[0]
predictedSentiments = [];

for i in sentimentIndices:
    predictedSentiments.append(sentiments[i]);
finalPredictedSentiments = pd.DataFrame(np.transpose(np.matrix(predictedSentiments)));
finalPredictedSentiments.columns=["Sentiment"]
finalResults = pd.concat([MongoReviews.getReviews().reset_index(), finalPredictedSentiments], axis=1)
# finalResults = finalResults.set_index("ReviewId")

finalResults.to_csv(os.getcwd() +"\\datasets\\finalPredictedSentiments.csv",
                                sep=',');
print(finalResults.values.tolist());
updateProducts = MongoReviews.putPredictions(finalResults.values.tolist());
print("Done with database updation, below are the updates: ");
print(updateProducts);

[['58e9efd543080021dc065ccc', 'HTC', 'good phone', 'positive'], ['58e9efd543080021dc065ccc', 'HTC', 'bad phone', 'negative'], ['58e9efd543080021dc065ccc', 'HTC', 'this is not a good phone', 'negative'], ['58e9efd543080021dc065ccc', 'HTC', 'i dont like this phone', 'negative'], ['58e9efd543080021dc065ccc', 'HTC', 'worst phone ever', 'negative'], ['58e9efd543080021dc065ccc', 'HTC', 'hate this phone', 'negative'], ['58e9efd543080021dc065ccc', 'HTC', 'dont buy this phone', 'negative'], ['58e9efd543080021dc065ccc', 'HTC', 'costly but good', 'neutral'], ['58e9efd543080021dc065ccc', 'HTC', 'never go with this', 'positive'], ['58e9efd543080021dc065ccc', 'HTC', 'best phone at this price range', 'positive'], ['58e9efd543080021dc065ccc', 'HTC', 'one of the worst phones', 'negative'], ['58e9efd543080021dc065ccd', 'Windows', 'dont buy this phone', 'negative'], ['58e9efd543080021dc065ccd', 'Windows', 'i dont like this phone', 'negative'], ['58e9efd543080021dc065ccd', 'Windows', 'good phone', 'positi